# Preprocesado

Técnicas de preprocesado empleadas:
1. **Lowercase:** Convertimos todo el texto a minúsculas para evitar repeticiones de palabras, esto puede ocurrir cuando ciertas funciones no son case sensitive
2. **Eliminar etiquetas html, direcciones email y urls:** Para hacer esto se han empleado expresiones regulares
3. **Eliminar signos de puntuación**
4. **POS Tagging:** Asignar una categoría a cada palabra (ejemplo: adjetivo, nombre, adverbio, verbo...). Esto es necesario para realizar Lemmatization con éxito.
5. **Lemmatization:** Recibe el tipo de palabra (obtenido en POS Tagging) como parámetro, en caso contrario considera a la palabra como un nombre, lo que afecta considerablemente a la utilidad de esta función.
6. **Stop Words Removal:** Elimina palabras innecesarias que generalmente hacen que los algoritmos tengan peor rendimiento. Ejemplos: Phrasal verbs, preposiciones, entre otros


La función POS-tagging devuelve más valores de los que se necesitan, mientras que la función de lematización solamente necesita nombre, adjetivo, adverbio y nombre. Para convertir de uno a otro usamos un map/diccionario.

In [1]:
pos_map = {
'CC': 'n','CD': 'n', 'DT': 'n','EX': 'n', 'FW': 'n','IN': 'n', 'JJ': 'a','JJR': 'a', 'JJS': 'a','LS': 'n', 'MD': 'v','NN': 'n',
'NNS': 'n','NNP': 'n', 'NNPS': 'n','PDT': 'n', 'POS': 'n','PRP': 'n', 'PRP$': 'r','RB': 'r', 'RBR': 'r','RBS': 'r', 'RP': 'n','TO': 'n',
'UH': 'n','VB': 'v', 'VBD': 'v','VBG': 'v', 'VBN': 'v','VBP': 'v', 'VBZ': 'v','WDT': 'n', 'WP': 'n','WP$': 'n', 'WRB': 'r'
}

Importamos las stop words del inglés

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

print('En inglés hay un total de ', len(stop_words), 'stopwords')

En inglés hay un total de  179 stopwords


Imprimimos las primeras 10 stopwords como ejemplo

In [3]:
list(stop_words)[0:10]

['now',
 'why',
 "isn't",
 'further',
 'once',
 'his',
 'about',
 'more',
 'be',
 'what']

### Lematización

In [4]:
from nltk.stem import WordNetLemmatizer

example_sent = "<HTML>This <p>is.a</p> ! jua@email.com sentences, showing off the <br> stop words filtration. http://www.youtube.com"
# Convertimos a minúscula (necesario para stopwords)
print('Frase inicial: ', example_sent, '\n')
example_sent = example_sent.lower()


# Eliminar etiquetas html, direcciones email y urls 
# HTML TAGS (al ser resultado de web scrapping, es conveniente asegurarse)
from bs4 import BeautifulSoup
example_sent = BeautifulSoup(example_sent, 'lxml').text

# EMAIL ADDRESSES
import re
example_sent = re.sub(r'[\w\.-]+@[\w\.-]+', ' ', example_sent)

# URLs
example_sent = re.sub(r'http\S+', '', example_sent)

# Eliminar signos de puntuación
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
word_tokens = tokenizer.tokenize(example_sent)
# Ahora tenemos las palabras tokenizadas sin signos de puntuación y con stop words
print('Frase sin direcciones de email, etiquetas html y urls', example_sent)

# Realizamos POS Tagging (eliminaremos las stopwords más adelante, ya que estas mejoran la precisión de POS tagging)
# Este método devuelve una lista de tuplas del tipo (palabra, categoría)
tags = nltk.pos_tag(word_tokens)
print('Palabras con sus tags asignados')
print('tags', tags)

lemmatizer = WordNetLemmatizer()
# Aplicamos lematización a todas las palabras del texto según su categoría
for i, word in enumerate(word_tokens):
    # Si la key no aparece en el mapa, se le considera un nombre, de esta forma se evitan errores de compilación
    word_tokens[i] = lemmatizer.lemmatize(word, pos=pos_map.get(tags[i][1] , 'n'))

# Quitamos las stop words
filtered_sentence = [w for w in word_tokens if not w in stop_words]

print('Frase tras aplicar preprocesado: ')
filtered_sentence

Frase inicial:  <HTML>This <p>is.a</p> ! jua@email.com sentences, showing off the <br> stop words filtration. http://www.youtube.com 

Frase sin direcciones de email, etiquetas html y urls this is.a !   sentences, showing off the  stop words filtration. 
Palabras con sus tags asignados
tags [('this', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('sentences', 'NNS'), ('showing', 'VBG'), ('off', 'RP'), ('the', 'DT'), ('stop', 'NN'), ('words', 'NNS'), ('filtration', 'NN')]
Frase tras aplicar preprocesado: 


['sentence', 'show', 'stop', 'word', 'filtration']

### Valores que devuelve POS Tagging

- **CC:**	coordinating conjunction
- **CD:**	cardinal digit
- **DT:**	determiner
- **EX:**	existential there (like: "there is" ... think of it like "there exists")
- **FW:**	foreign word
- **IN:**	preposition/subordinating conjunction
- **JJ:**	adjective	'big'
- **JJR:**	adjective, comparative	'bigger'
- **JJS:**	adjective, superlative	'biggest'
- **LS:**	list marker	1)
- **MD:**	modal	could, will
- **NN:**	noun, singular 'desk'
- **NNS:**	noun plural	'desks'
- **NNP:**	proper noun, singular	'Harrison'
- **NNPS:**	proper noun, plural	'Americans'
- **PDT:**	predeterminer	'all the kids'
- **POS:** possessive ending	parent\'s
- **PRP:**	personal pronoun	I, he, she
- **PRP\$:**	possessive pronoun	my, his, hers
- **RB:**	adverb	very, silently,
- **RBR:**	adverb, comparative	better
- **RBS:**	adverb, superlative	best
- **RP:**	particle	give up
- **TO:**	to	go 'to' the store.
- **UH:**	interjection	errrrrrrrm
- **VB:**	verb, base form	take
- **VBD:**	verb, past tense	took
- **VBG:**	verb, gerund/present participle	taking
- **VBN:**	verb, past participle	taken
- **VBP:**	verb, sing. present, non-3d	take
- **VBZ:**	verb, 3rd person sing. present	takes
- **WDT:**	wh-determiner	which
- **WP:**	wh-pronoun	who, what
- **WP\$:**	possessive wh-pronoun	whose
- **WRB:**	wh-abverb	where, when

### Categorías que recibe Lemmatization
ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'

## Convertir el preprocesado en una función

In [5]:
from bs4 import BeautifulSoup
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def tokenizer(example_sent):

    example_sent = example_sent.lower()
    
    # HTML TAGS
    example_sent = BeautifulSoup(example_sent, 'lxml').text

    # EMAIL ADDRESSES
    example_sent = re.sub(r'[\w\.-]+@[\w\.-]+', ' ', example_sent)

    # URLs
    example_sent = re.sub(r'http\S+', '', example_sent)

    # Signos de puntuación
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(example_sent)

    # POS Tagging 
    tags = nltk.pos_tag(word_tokens)

    # Lemmatization
    for i, word in enumerate(word_tokens):
        word_tokens[i] = lemmatizer.lemmatize(word, pos=pos_map.get(tags[i][1] , 'n'))

    # stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    return filtered_sentence

Probamos la función:

In [6]:
test = "<HTML>This <p>is.a</p> ! jua@email.com sentences, showing off the <br> stop words filtration. http://www.youtube.com"
tokenizer(test)

['sentence', 'show', 'stop', 'word', 'filtration']